In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
cd

In [ ]:
# import shutil
# source = '../input/unet-model/'
# destination = '/root/'
# shutil.move(source, destination) 

In [ ]:
# import os
# import sys
# # insert at 1, 0 is the script path (or '' in REPL)
# sys.path.insert(1, '../input/filetoimport')

# import model_all
# # os.chdir("../working/")

In [ ]:
# import os
# import sys
# # insert at 1, 0 is the script path (or '' in REPL)
# sys.path.insert(1, '../input/unet-model')
# # import modelall
# import modelall as model_all
# # os.chdir("../working/")

In [ ]:
# import os
# os.chdir("../input/unet-model/")
# import modellall.py

In [ ]:
import nibabel as nib
import os
import torch
import torchvision.transforms as T

# Data original source downloaded from : https://zenodo.org/record/3757476#.Xp0FhB9fgUE
# Same data in Kaggle  : https://www.kaggle.com/andrewmvd/covid19-ct-scans

# Data and mask path
data_path = "/kaggle/input/covid19-ct-scans/ct_scans/coronacases_org_001.nii"
mask_path = "/kaggle/input/covid19-ct-scans/infection_mask/coronacases_001.nii"

# Lung window in HU units
HU_min = -1000
HU_max = 400

# names of files in data folder
# arr = os.listdir(data_path)

# select train and val subjects
train_data_numbers = range(0, 0)  # data from coronacases.org
val_data_numbers = range(0, 1)  # data from radiopedia.org

# Resizing function for image and mask according to the model input dimension
# resize_image = T.Resize(size=(572, 572))


# function to preprocess data
def data_preprocess(path, hu_min, hu_max):
    volume_data = nib.load(path)  # load data
    volume_data_numpy = volume_data.get_fdata()  # get data as numpy
    volume_data_tensor = torch.tensor(volume_data_numpy)  # convert to torch tensor
    volume_data_tensor_clamped = torch.clamp(volume_data_tensor, min=hu_min, max=hu_max)  # apply HU lung window
    volume_data_tensor_clamped_normalized = (volume_data_tensor_clamped-hu_min) / (hu_max-hu_min)  # normalize to [0,1]
    return volume_data_tensor_clamped_normalized


# function to obtain maask
def mask_obtain(fpath):
    mask = nib.load(fpath)  # load mask
    mask_numpy = mask.get_fdata()  # get mask as numpy
    mask_tensor = torch.tensor(mask_numpy)  # convert to torch tensor
    return mask_tensor


# add zero padding if size of image less than required input size
def padding_size(slices):
    if (572-slices.size()[1]) % 2 == 0:  # See if the difference between required size and data size is even or odd
        # if difference is even, pad same number to either side
        pad1 = (572-slices.size()[1]) // 2
        pad2 = (572-slices.size()[1]) // 2
    else:
        # if difference is even, pad one side one value more than other
        pad1 = (572-slices.size()[1]) // 2
        pad2 = ((572-slices.size()[1]) // 2)+1

    if (572-slices.size()[2]) % 2 == 0:  # See if the difference between required size and data size is even or odd
        # if difference is even, pad same number to either side
        pad3 = (572-slices.size()[2]) // 2
        pad4 = (572-slices.size()[2]) // 2
    else:
        # if difference is even, pad one side one value more than other
        pad3 = (572-slices.size()[2]) // 2
        pad4 = ((572-slices.size()[2]) // 2)+1

    return [pad4, pad3, pad2, pad1]  # return the number of zero padding in each side of the slice

In [ ]:
# test = data_preprocess(data_path,-1000,400)
# test.size()

In [ ]:
# # validation data
# Initialize for stacking
val_data = torch.empty((1, 572, 572))
val_label = torch.empty((1, 572, 572))

# Function to form train data and train label
for i in val_data_numbers:
    file_path = data_path  # Path to validation data
    data = data_preprocess(file_path, HU_min, HU_max)  # Preprocess the data
    data = data.permute(2, 0, 1)  # change the dimension (H,W,C) ---> (C,H,W) , since ConstantPad2d works with this config
    P = padding_size(data)  # Obtain padding sizes
    data = torch.nn.ConstantPad2d(P, 0)(data)  # zero pad the slices
    val_data = torch.cat((val_data, data), 0)  # stack the data along dimension C

    file_path_mask = mask_path
    label = mask_obtain(file_path_mask)  # Path to mask
    label = label.permute(2, 0, 1)  # change the dimension (H,W,C) ---> (C,H,W) , since ConstantPad2d works with this config
    label = torch.nn.ConstantPad2d(P, 0)(label)  # zero pad mask
    val_label = torch.cat((val_label, label), 0)  # stack the masks along dimension C

# remove the empty
val_data = val_data[1:val_data.size()[0], :, :]
val_label = val_label[1:val_label.size()[0], :, :]

# Determine which slices are all black
idx = []
for i in range(val_label.size()[0]):
    img_max = torch.max(val_label[i, :, :])
    if img_max == 1:
        idx.append(i)  # having white regions

# Choose data without completely black mask, i.e, having atleast some white segmented region
val_data_new = val_data[idx, :, :]
val_label_new = val_label[idx, :, :]

# (C,H,W) ---> (H,W,C) since Dataset class has this config (this part is not necessary if we change the config of
# Dataset class)
val_data_new = val_data_new.permute(1, 2, 0)
val_label_new = val_label_new.permute(1, 2, 0)

In [ ]:
val_label_new.size()

In [ ]:
import torch.nn.functional as F


def DCE(inputs, targets, smooth=1):
    inputs = torch.sigmoid(inputs)

    # flatten label and prediction tensors
    inputs = inputs.view(-1)
#     print(inputs.size())
#     print(targets.size())
    targets = targets.view(-1)

    intersection = (inputs * targets).sum()
    dice = (2. * intersection+smooth) / (inputs.sum()+targets.sum()+smooth)
    return 1-dice


def GDCE(inputs, targets, smooth=1):
    inputs = F.sigmoid(inputs)

    # flatten label and prediction tensors
    inputs = inputs.view(-1)
    targets = targets.view(-1)

    # targets and corresponding predictions of class 1
    idx1 = (targets == 0)
    T1 = targets[idx1]
    P1 = inputs[idx1]

    # targets and corresponding predictions of class 2
    idx2 = (targets == 1)
    T2 = targets[idx2]
    P2 = inputs[idx2]

    # Weights for each class
    W1 = 1 / (len(T1) * len(T1))
    W2 = 1 / (len(T2) * len(T2))

    # Numerator and denominator of generalized dice loss
    NR = W1 * (T1 * P1).sum()+W2 * (T2 * P2).sum()
    DR = W1 * (T1+P1).sum()+W2 * (T2+P2).sum()

    loss = 1-(((2 * NR) + smooth)/ (DR+smooth))

    return loss

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Module

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Two sequential convolution section of Unet
def doubleconv(inp, out):
    double_conv = nn.Sequential(
        nn.Conv2d(inp, out, kernel_size=3),
        nn.BatchNorm2d(out,track_running_stats=False),
        nn.ReLU(inplace=True),
        nn.Conv2d(out, out, kernel_size=3),
        nn.BatchNorm2d(out,track_running_stats=False),
        nn.ReLU(inplace=True)
    )
    return double_conv


# Crop the encoder feature to the size of corresponding decoder for concatenation
def crop_feat1(input_tensor, target_tensor):
    out_size = target_tensor.size()[2]
    inp_size = input_tensor.size()[2]
    delta = (inp_size-out_size) // 2
    if (inp_size-out_size) % 2 == 0:
        result = input_tensor[:, :, delta:inp_size-delta, delta:inp_size-delta]
    else:
        result = input_tensor[:, :, delta:inp_size-delta-1, delta:inp_size-delta-1]

    return result


# Spatial Channel Attention Block
class sca(Module):
    def __init__(self, inp):
        super(sca, self).__init__()
        self.c_attn_conv = nn.Sequential(nn.Conv2d(inp, inp // 16, 1, bias=False),
                                         nn.ReLU(),
                                         nn.Conv2d(inp // 16, inp, 1, bias=False)
                                         )
        self.c_sig = nn.Sigmoid()

        self.avg_ch = nn.AdaptiveAvgPool2d(1)
        self.max_ch = nn.AdaptiveMaxPool2d(1)
        self.s_attn_conv = nn.Sequential(nn.Conv2d(2, 1, kernel_size=7, padding=7 // 2, bias=False),
                                         nn.Sigmoid())

    def forward(self, input_tensor):
        # Channel Attention
        avg_ch_pool = self.avg_ch(input_tensor)
        max_ch_pool = self.max_ch(input_tensor)

        out_1 = self.c_attn_conv(avg_ch_pool)
        out_2 = self.c_attn_conv(max_ch_pool)

        c_sum = out_1+out_2
        ch_out = self.c_sig(c_sum)
        input_tensor = input_tensor * ch_out
        # Spatial Attention
        avg_pool = torch.mean(input_tensor, dim=1, keepdim=True)
        max_pool = torch.max(input_tensor, dim=1, keepdim=True)
        x = torch.cat([avg_pool, max_pool.values], dim=1)

        x = self.s_attn_conv(x)
        x = torch.mul(input_tensor, x)

        return x


# Function to apply spatial channel attention block
def spatial_channel_attn(input_tensor):
    input_tensor = input_tensor.type(torch.cuda.FloatTensor)
    inp = input_tensor.size()[1]

    sca_model = sca(inp).to(device)
    x = sca_model(input_tensor)

    return x


# Atrous spatial pyramid pooling block
class aspp(Module):
    def __init__(self, inp, out):
        super(aspp, self).__init__()
        self.aconv0 = nn.Sequential(nn.Conv2d(inp, out, kernel_size=3, stride=1, padding=4, dilation=4, bias=False),
                                    nn.BatchNorm2d(out,track_running_stats=False),
                                    nn.ReLU(inplace=True))
        self.aconv1 = nn.Sequential(nn.Conv2d(inp, out, kernel_size=3, stride=1, padding=6, dilation=6, bias=False),
                                    nn.BatchNorm2d(out,track_running_stats=False),
                                    nn.ReLU(inplace=True))
        self.aconv2 = nn.Sequential(nn.Conv2d(inp, out, kernel_size=3, stride=1, padding=12, dilation=12, bias=False),
                                    nn.BatchNorm2d(out,track_running_stats=False),
                                    nn.ReLU(inplace=True))
        self.aconv3 = nn.Sequential(nn.Conv2d(inp, out, kernel_size=3, stride=1, padding=18, dilation=18, bias=False),
                                    nn.BatchNorm2d(out,track_running_stats=False),
                                    nn.ReLU(inplace=True))
        self.aconv4 = nn.Sequential(nn.Conv2d(inp, out, kernel_size=3, stride=1, padding=24, dilation=24, bias=False),
                                    nn.BatchNorm2d(out,track_running_stats=False),
                                    nn.ReLU(inplace=True))
        self.final_conv = nn.Sequential(
            nn.Conv2d(out * 5, inp, kernel_size=1, stride=1, padding=0, dilation=1, bias=False),
            nn.BatchNorm2d(inp),
            nn.ReLU(inplace=True))

    def forward(self, input_tensor):
        x0 = self.aconv0(input_tensor)
        x1 = self.aconv1(input_tensor)
        x2 = self.aconv2(input_tensor)
        x3 = self.aconv3(input_tensor)
        x4 = self.aconv4(input_tensor)
        x = torch.cat((x0, x1, x2, x3, x4), dim=1)
        aspp_out = self.final_conv(x)

        return aspp_out


# Function to carry out atrous spatial pyramid pooling (In our case input and output is made to be of same size)
def atrous_spatial_pyramid_pooling(input_tensor):
    input_tensor = input_tensor.type(torch.cuda.FloatTensor)
    inp = input_tensor.size()[1]
    out = inp // 4

    asppmodel = aspp(inp, out).to(device)
    aspp_out = asppmodel(input_tensor)
    res_aspp = aspp_out+input_tensor
    return res_aspp


# Unet Model
class Unet(Module):
    def __init__(self):
        super(Unet, self).__init__()

        self.maxpool2x2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.DownConv1 = doubleconv(1, 64)
        self.DownConv2 = doubleconv(64, 128)
        self.DownConv3 = doubleconv(128, 256)
        self.DownConv4 = doubleconv(256, 512)
        self.DownConv5 = doubleconv(512, 1024)

        self.UpTrans1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.UpTrans2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.UpTrans3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.UpTrans4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        self.UpConv1 = doubleconv(1024, 512)
        self.UpConv2 = doubleconv(512, 256)
        self.UpConv3 = doubleconv(256, 128)
        self.UpConv4 = doubleconv(128, 64)

        self.out1 = nn.Conv2d(64, 1, kernel_size=1)
        self.out2 = nn.Conv2d(128, 1, kernel_size=1)
        self.out3 = nn.Conv2d(256, 1, kernel_size=1)
        self.out4 = nn.Conv2d(512, 1, kernel_size=1)

    def forward(self, x):
        # x=(batch size, channel, height, width)
        # encoder

        x1 = self.DownConv1(x)
        x2 = self.maxpool2x2(x1)
        x3 = self.DownConv2(x2)
        x4 = self.maxpool2x2(x3)
        x5 = self.DownConv3(x4)
        x6 = self.maxpool2x2(x5)
        x7 = self.DownConv4(x6)
        x8 = self.maxpool2x2(x7)
        x9 = self.DownConv5(x8)

        # ASPP
        x9_aspp = atrous_spatial_pyramid_pooling(x9)

        # decoder
        z1 = self.UpTrans1(spatial_channel_attn(x9_aspp))
        y1 = crop_feat1(x7, z1)
        x10 = self.UpConv1(torch.cat([y1, z1], 1))

        z2 = self.UpTrans2(spatial_channel_attn(x10))
        y2 = crop_feat1(x5, z2)
        x11 = self.UpConv2(torch.cat([y2, z2], 1))

        z3 = self.UpTrans3(spatial_channel_attn(x11))
        y3 = crop_feat1(x3, z3)
        x12 = self.UpConv3(torch.cat([y3, z3], 1))

        z4 = self.UpTrans4(spatial_channel_attn(x12))
        y4 = crop_feat1(x1, z4)
        x13 = self.UpConv4(torch.cat([y4, z4], 1))

        # ASPP
        x13_aspp = atrous_spatial_pyramid_pooling(x13)
        out1 = self.out1(x13_aspp)
        out2 = self.out2(x12)
        out3 = self.out3(x11)
        out4 = self.out4(x10)
        return out1, out2, out3, out4

# %%%%%%% model  check %%%%%%%%%

# if __name__ == "__main__":
#     image = torch.rand((2, 1, 572, 572))
#     model = Unet()
#     model.to(device)
#     image=image.to(device)
#     y = model(image)

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms.functional as F
# from data_preparation import train_data_new, train_label_new, val_data_new, val_label_new
from torch.optim import Adam
import torchvision.transforms as T
import warnings
from torch.cuda import amp
from tqdm import tqdm
# from loss import DCE
import matplotlib.pyplot as plt

# Ignore warnings
warnings.filterwarnings("ignore")

# Create GradScaler for mixed precision training
scaler = amp.GradScaler()

# Choose the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Function to crop mask to the size of output of U-net
def crop_feat(inp):
    inp_size = inp.size()[2]  # here it is 572 as decided during data preparation
    delta = (inp_size-388) // 2
    return inp[:, delta:inp_size-delta, delta:inp_size-delta]


# Function to augment data
def data_augmentation(image, mask):
    # Horizontal flip
    if torch.rand(1) > 0.5:
        image = F.hflip(image)
        mask = F.hflip(mask)
    # Vertical flip
    if torch.rand(1) > 0.5:
        image = F.vflip(image)
        mask = F.vflip(mask)
    # Rotate 90 degree or -90 degree
    if torch.rand(1) > 0.5:
        if torch.rand(1) > 0.5:
            image = torch.rot90(image, 1, [0, 1])
            mask = torch.rot90(mask, 1, [0, 1])
        else:
            image = torch.rot90(image, -1, [0, 1])
            mask = torch.rot90(mask, -1, [0, 1])
    return image, mask


# Dataset class
class CovidSegData(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.transform = transform
        self.labels = labels

    def __len__(self):
        return self.labels.size()[2]  # Data in (H,W,C) format obtained from data preparation

    def __getitem__(self, item):
        # Select each slice
        slices = self.data[:, :, item]
        masks = self.labels[:, :, item]
        if self.transform is not None:  # if transform is True carry out data augmentation
            slices, masks = data_augmentation(slices, masks)
        return slices, masks


# Loss function
def loss_function(preds1, preds2, preds3, preds4, GT):
    # resize groundtruth according to the predicted label dimensions
    GT1a = GT  # Ground truth already cropped to the output dimension of Unet
    resize_mask2 = T.Resize(size=(preds2.size()[2], preds2.size()[3]))
    GT2 = resize_mask2(GT)
    resize_mask3 = T.Resize(size=(preds3.size()[2], preds3.size()[3]))
    GT3 = resize_mask3(GT)
    resize_mask4 = T.Resize(size=(preds4.size()[2], preds4.size()[3]))
    GT4 = resize_mask4(GT)
    # Dice score for each predictions
    D1 = DCE(preds1, GT1a)
    D2 = DCE(preds2, GT2)
    D3 = DCE(preds3, GT3)
    D4 = DCE(preds4, GT4)
    # weighted loss -- deep supervision
    loss_final = (0.5 * D1)+(0.2 * D2)+(0.1 * D3)+(0.1 * D4)
    return loss_final

In [ ]:
val_dataset = CovidSegData(val_data_new, val_label_new, transform=False)

val_loader = DataLoader(val_dataset, batch_size=4,num_workers=2, shuffle=False, pin_memory=True)


In [ ]:
# Initialize network
model = Unet()
losses=0
i =0
model.load_state_dict(torch.load('/kaggle/input/50epoches/all_weights/weights1 (1).pth'), strict=False)
# model = (torch.load('/kaggle/input/k-fold-weights/weights1.pth'))
model.to(device)  # Move the model to GPU
# Optimizer and Scheduler
optimizer = Adam(model.parameters(), lr=2e-4, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
num_epochs = 1  # number of epochs
for epoch in range(num_epochs):
    # Model evaluation - validation mode
    model.eval()  # Set model in evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=True)  # For progress bar
        for batch_idx, (images, GT1) in loop:
            dataset_size = 0
            running_loss = 0.0
            # Crop mask to the size of output of Unet,i.e, 388 in this case
            GT = crop_feat(GT1)
            # Get data to cuda
            images = images.to(device)
            GT = GT.to(device)
            batch_size = images.size()[0]  # Size of each batch
            images = torch.unsqueeze(images, 1)  # get correct input dimensions
            images = images.type(torch.cuda.FloatTensor)  # Convert input to Float tensor
            preds1, preds2, preds3, preds4 = model(images)  # predictions
            loss = loss_function(preds1, preds2, preds3, preds4, GT)  # loss
           
            
            # Epoch loss calculation
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            losses= losses+ epoch_loss
#             print(epoch_loss)
            i = i+1
#             # Update progress bar
            loop.set_postfix(loss=loss.item(), epoch_loss=epoch_loss)
            
            # Visualize image, mask and predicted mask
            # Pass the prediction through sigmoid to get probabilities and threshold at 0.5 to obtain binary image
#             print(torch.max(torch.sigmoid(preds1)),torch.min(torch.sigmoid(preds1)) )
            
            z = preds1
            z[z<0.1] = 0
#             torch.sigmoid(preds1)*(10**9) > 0.5
            z1 = z[0].detach().cpu()
            z2 = z1.squeeze()
            # Input Slice
            y1 = images[0].detach().cpu()
            y = crop_feat(y1).squeeze()
            # Mask or Ground truth
            p1 = GT[0].detach().cpu().squeeze()

#             Figure with input image, groundtrith and predicted mask
#             f, axarr = plt.subplots(1, 3)
#             axarr[0].imshow(y, cmap='gray')
#             axarr[1].imshow(p1, cmap='gray')
#             axarr[2].imshow(z2, cmap='gray')
#             plt.show()
print(losses/i)

In [ ]:
losses=0
i =0
# Initialize network
model = Unet()
model.load_state_dict(torch.load('/kaggle/input/50epoches/all_weights/weights2 (1).pth'), strict=False)
# model = (torch.load('/kaggle/input/k-fold-weights/weights1.pth'))
model.to(device)  # Move the model to GPU
# Optimizer and Scheduler
optimizer = Adam(model.parameters(), lr=2e-4, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
num_epochs = 1  # number of epochs
for epoch in range(num_epochs):
    # Model evaluation - validation mode
    model.eval()  # Set model in evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=True)  # For progress bar
        for batch_idx, (images, GT1) in loop:
            dataset_size = 0
            running_loss = 0.0
            # Crop mask to the size of output of Unet,i.e, 388 in this case
            GT = crop_feat(GT1)
            # Get data to cuda
            images = images.to(device)
            GT = GT.to(device)

            batch_size = images.size()[0]  # Size of each batch

            images = torch.unsqueeze(images, 1)  # get correct input dimensions

            images = images.type(torch.cuda.FloatTensor)  # Convert input to Float tensor

            preds1, preds2, preds3, preds4 = model(images)  # predictions
            loss = loss_function(preds1, preds2, preds3, preds4, GT)  # loss
            # Epoch loss calculation
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            losses= losses+ epoch_loss
            i = i+1
            # Update progress bar
            loop.set_postfix(loss=loss.item(), epoch_loss=epoch_loss)
            # Visualize image, mask and predicted mask
            # Pass the prediction through sigmoid to get probabilities and threshold at 0.5 to obtain binary image
#             print(torch.max(torch.sigmoid(preds1)),torch.min(torch.sigmoid(preds1)) )
            
#             z = torch.sigmoid(preds1)*(10**9) > 0.5
            z = preds1
            z[z<0.1] = 0
            z1 = z[0].detach().cpu()
            z2 = z1.squeeze()
            # Input Slice
            y1 = images[0].detach().cpu()
            y = crop_feat(y1).squeeze()
            # Mask or Ground truth
            p1 = GT[0].detach().cpu().squeeze()

            # Figure with input image, groundtrith and predicted mask
#             f, axarr = plt.subplots(1, 3)
#             axarr[0].imshow(y, cmap='gray')
#             axarr[1].imshow(p1, cmap='gray')
#             axarr[2].imshow(z2, cmap='gray')
#             plt.show()
print(losses/i)

In [ ]:
# Initialize network
losses=0
i =0
model = Unet()
model.load_state_dict(torch.load('/kaggle/input/50epoches/all_weights/weights3 (1).pth'), strict=False)
# model = (torch.load('/kaggle/input/k-fold-weights/weights1.pth'))
model.to(device)  # Move the model to GPU
# Optimizer and Scheduler
optimizer = Adam(model.parameters(), lr=2e-4, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
num_epochs = 1  # number of epochs
for epoch in range(num_epochs):
    # Model evaluation - validation mode
    model.eval()  # Set model in evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=True)  # For progress bar
        for batch_idx, (images, GT1) in loop:
            dataset_size = 0
            running_loss = 0.0
            # Crop mask to the size of output of Unet,i.e, 388 in this case
            GT = crop_feat(GT1)
            # Get data to cuda
            images = images.to(device)
            GT = GT.to(device)

            batch_size = images.size()[0]  # Size of each batch

            images = torch.unsqueeze(images, 1)  # get correct input dimensions

            images = images.type(torch.cuda.FloatTensor)  # Convert input to Float tensor

            preds1, preds2, preds3, preds4 = model(images)  # predictions
            loss = loss_function(preds1, preds2, preds3, preds4, GT)  # loss
            # Epoch loss calculation
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            losses= losses+ epoch_loss
            i = i+1
            # Update progress bar
            loop.set_postfix(loss=loss.item(), epoch_loss=epoch_loss)
            # Visualize image, mask and predicted mask
            # Pass the prediction through sigmoid to get probabilities and threshold at 0.5 to obtain binary image
#             print(torch.max(torch.sigmoid(preds1)),torch.min(torch.sigmoid(preds1)) )
            
#             z = torch.sigmoid(preds1)*(10**9) > 0.5
            z = preds1
            z[z<0.1] = 0
            z1 = z[0].detach().cpu()
            z2 = z1.squeeze()
            # Input Slice
            y1 = images[0].detach().cpu()
            y = crop_feat(y1).squeeze()
            # Mask or Ground truth
            p1 = GT[0].detach().cpu().squeeze()

            # Figure with input image, groundtrith and predicted mask
#             f, axarr = plt.subplots(1, 3)
#             axarr[0].imshow(y, cmap='gray')
#             axarr[1].imshow(p1, cmap='gray')
#             axarr[2].imshow(z2, cmap='gray')
#             plt.show()
print(losses/i)

In [ ]:
losses=0
i =0
# Initialize network
model = Unet()
model.load_state_dict(torch.load('/kaggle/input/50epoches/all_weights/weights4 (1).pth'), strict=False)
# model = (torch.load('/kaggle/input/k-fold-weights/weights1.pth'))
model.to(device)  # Move the model to GPU
# Optimizer and Scheduler
optimizer = Adam(model.parameters(), lr=2e-4, weight_decay=1e-6)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
num_epochs = 1  # number of epochs
for epoch in range(num_epochs):
    # Model evaluation - validation mode
    model.eval()  # Set model in evaluation mode
    with torch.no_grad():  # Disable gradient calculation
        loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=True)  # For progress bar
        for batch_idx, (images, GT1) in loop:
            dataset_size = 0
            running_loss = 0.0
            # Crop mask to the size of output of Unet,i.e, 388 in this case
            GT = crop_feat(GT1)
            # Get data to cuda
            images = images.to(device)
            GT = GT.to(device)

            batch_size = images.size()[0]  # Size of each batch

            images = torch.unsqueeze(images, 1)  # get correct input dimensions

            images = images.type(torch.cuda.FloatTensor)  # Convert input to Float tensor

            preds1, preds2, preds3, preds4 = model(images)  # predictions
            loss = loss_function(preds1, preds2, preds3, preds4, GT)  # loss
            # Epoch loss calculation
            running_loss += (loss.item() * batch_size)
            dataset_size += batch_size
            epoch_loss = running_loss / dataset_size
            losses= losses+ epoch_loss
            i = i+1
            # Update progress bar
            loop.set_postfix(loss=loss.item(), epoch_loss=epoch_loss)
            # Visualize image, mask and predicted mask
            # Pass the prediction through sigmoid to get probabilities and threshold at 0.5 to obtain binary image
#             print(torch.max(torch.sigmoid(preds1)),torch.min(torch.sigmoid(preds1)) )
            
#             z = torch.sigmoid(preds1)*(10**9) > 0.5
            z = preds1
            z[z<0.1] = 0
            z1 = z[0].detach().cpu()
            z2 = z1.squeeze()
            # Input Slice
            y1 = images[0].detach().cpu()
            y = crop_feat(y1).squeeze()
            # Mask or Ground truth
            p1 = GT[0].detach().cpu().squeeze()

#             Figure with input image, groundtrith and predicted mask
#             f, axarr = plt.subplots(1, 3)
#             axarr[0].imshow(y, cmap='gray')
#             axarr[1].imshow(p1, cmap='gray')
#             axarr[2].imshow(z2, cmap='gray')
#             plt.show()
print(losses/i)

In [ ]:
# losses=0
# i =0
# # Initialize network
# model = Unet()
# model.load_state_dict(torch.load('/kaggle/input/50epoches/all_weights/weights5 (1).pth'), strict=False)
# # model = (torch.load('/kaggle/input/k-fold-weights/weights1.pth'))
# model.to(device)  # Move the model to GPU
# # Optimizer and Scheduler
# optimizer = Adam(model.parameters(), lr=2e-4, weight_decay=1e-6)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)
# num_epochs = 1  # number of epochs
# for epoch in range(num_epochs):
#     # Model evaluation - validation mode
#     model.eval()  # Set model in evaluation mode
#     with torch.no_grad():  # Disable gradient calculation
#         loop = tqdm(enumerate(val_loader), total=len(val_loader), leave=True)  # For progress bar
#         for batch_idx, (images, GT1) in loop:
#             dataset_size = 0
#             running_loss = 0.0
#             # Crop mask to the size of output of Unet,i.e, 388 in this case
#             GT = crop_feat(GT1)
#             # Get data to cuda
#             images = images.to(device)
#             GT = GT.to(device)

#             batch_size = images.size()[0]  # Size of each batch

#             images = torch.unsqueeze(images, 1)  # get correct input dimensions

#             images = images.type(torch.cuda.FloatTensor)  # Convert input to Float tensor

#             preds1, preds2, preds3, preds4 = model(images)  # predictions
#             loss = loss_function(preds1, preds2, preds3, preds4, GT)  # loss
#             # Epoch loss calculation
#             running_loss += (loss.item() * batch_size)
#             dataset_size += batch_size
#             epoch_loss = running_loss / dataset_size
#             losses= losses+ epoch_loss
#             i = i+1
#             # Update progress bar
#             loop.set_postfix(loss=loss.item(), epoch_loss=epoch_loss)
#             # Visualize image, mask and predicted mask
#             # Pass the prediction through sigmoid to get probabilities and threshold at 0.5 to obtain binary image
# #             print(torch.max(torch.sigmoid(preds1)),torch.min(torch.sigmoid(preds1)) )
            
# #             z = torch.sigmoid(preds1)*(10**9) > 0.5
#             z = preds1
#             z[z<0.1] = 0
#             z1 = z[0].detach().cpu()
#             z2 = z1.squeeze()
#             # Input Slice
#             y1 = images[0].detach().cpu()
#             y = crop_feat(y1).squeeze()
#             # Mask or Ground truth
#             p1 = GT[0].detach().cpu().squeeze()

#             # Figure with input image, groundtrith and predicted mask
# #             f, axarr = plt.subplots(1, 3)
# #             axarr[0].imshow(y, cmap='gray')
# #             axarr[1].imshow(p1, cmap='gray')
# #             axarr[2].imshow(z2, cmap='gray')
# #             plt.show()
# print(losses/i)